### Manufacturer Y Value Summary

Y value includes 6 indicators: nr_viewer_buyer, nr_buyers_sent_message, nr_messages_received_from_buyers, nr_brand_watchlist, nr_brand_supplierlist, pv_countwatch follow and supplier follow. 

* nr_brand_watchlist: the number of followed manufacturer on brand watchlist  
* nr_brand_supplierlist: the number of followed manufacturer on supplierlist  
* nr_messages_received_from_buyers: the number of manufacturer received message coming from buyer  
* nr_buyers_sent_message: the number of buyer who sent the message to manufacturer    
* pv_countwatch: manufacturer pageview number   
* nr_viewer_buyer: number of distinct buyer who viewed manufacturer  

In [19]:
import numpy as np
import pandas as pd
import math
import match_helper as helper
from imp import reload
reload(helper)
dbaccess = helper.get_dbaccess()
buyer_access = dbaccess + ['buyer']
manufacturer_access = dbaccess + ['manufacturer']
shared_access = dbaccess + ['shared']
crm_access = dbaccess + ['crm']
logging_access = dbaccess + ['logging']
messaging_access = dbaccess + ['messaging']
auth_access = dbaccess + ['auth']

/Users/glen/Desktop/Foursource/work/Manufacturer analysis/


#### brand watch follow and supplier follow

In [2]:
reload(helper)
brandfollow_sql = """
WITH follow AS (

SELECT    
          'manufacturer_watchlist-2-' || id_follow_company_brand AS uid_follow
        , 'manufacturer_watchlist' AS follow_type  
        , id_follow_company_brand AS id_follow_relation
        , 'Manufacturer' AS follower_entity_type
        , 2 AS follower_fk_entity_type
	, fk_manufacturer AS follower_id
	, 'Brand' AS followed_entity_type
	, 1 AS followed_fk_entity_type
	, fk_company_brand AS followed_id
	, active
	, created_at AS follow_timestamp
	, to_char(created_at::timestamp with time zone, 'iyyy"W"IW'::text) AS follow_year_week
	, to_char(created_at::timestamp with time zone, 'yyyy"M"mm'::text) AS follow_year_month
	, to_char(created_at::timestamp with time zone, 'yyyy'::text) AS follow_year
	, user_hashed_id AS follow_user_hash
FROM public.follow_company_brand

UNION

SELECT    
          'brand_watchlist-2-' || id_follow_manufacturer AS uid_follow
        , 'brand_watchlist' AS follow_type  
        , id_follow_manufacturer AS id_follow_relation
        , 'Brand' AS follower_entity_type
        , 1 AS follower_fk_entity_type
	, fk_company_brand AS follower_id
	, 'Manufacturer' AS followed_entity_type
	, 2 AS followed_fk_entity_type
	, fk_manufacturer AS followed_id
	, active
	, created_at AS follow_timestamp
	, to_char(created_at::timestamp with time zone, 'iyyy"W"IW'::text) AS follow_year_week
	, to_char(created_at::timestamp with time zone, 'yyyy"M"mm'::text) AS follow_year_month
	, to_char(created_at::timestamp with time zone, 'yyyy'::text) AS follow_year
	, user_hashed_id AS follow_user_hash
FROM public.follow_manufacturer

UNION 

SELECT
          'supplier-1-' || id_add_supplier AS uid_follow
        , 'supplier' AS follow_type  
        , id_add_supplier AS id_follow_relation
        , 'Brand' AS follower_entity_type
        , 1 AS follower_fk_entity_type
	, ext_id_company_brand AS follower_id
	, 'Manufacturer' AS followed_entity_type
	, 2 AS followed_fk_entity_type
	, ext_id_manufacturer AS followed_id
	, active
	, created_at AS follow_timestamp
	, to_char(created_at::timestamp with time zone, 'iyyy"W"IW'::text) AS follow_year_week
	, to_char(created_at::timestamp with time zone, 'yyyy"M"mm'::text) AS follow_year_month
	, to_char(created_at::timestamp with time zone, 'yyyy'::text) AS follow_year
	, NULL AS follow_user_hash
FROM public.add_supplier

UNION 

SELECT
          'customer-2-' || id_add_customer AS uid_follow
        , 'customer' AS follow_type  
        , id_add_customer AS id_follow_relation
        , 'Manufacturer' AS follower_entity_type
        , 2 AS follower_fk_entity_type
	, ext_id_manufacturer AS follower_id
	, 'Brand' AS followed_entity_type
	, 1 AS followed_fk_entity_type
	, ext_id_company_brand AS followed_id
	, active
	, created_at AS follow_timestamp
	, to_char(created_at::timestamp with time zone, 'iyyy"W"IW'::text) AS follow_year_week
	, to_char(created_at::timestamp with time zone, 'yyyy"M"mm'::text) AS follow_year_month
	, to_char(created_at::timestamp with time zone, 'yyyy'::text) AS follow_year
	, NULL AS follow_user_hash
FROM public.add_customer
ORDER BY follow_timestamp, id_follow_relation) 



SELECT f.followed_id AS manufacturer_id,
       COUNT( DISTINCT CASE WHEN follow_type = 'brand_watchlist' THEN f.follower_id ELSE null END) AS brandcount_watchlist,
       COUNT( DISTINCT CASE WHEN follow_type = 'supplier' THEN f.follower_id ELSE null END) AS brandcount_supplierlist

FROM follow AS f
WHERE f.follower_entity_type = 'Brand' AND f.followed_entity_type = 'Manufacturer' AND f.active = true
GROUP BY f.followed_id;
"""
b_df = helper.query_retrieval(brandfollow_sql, shared_access)
b_df[:5]

,manufacturer_id,brandcount_watchlist,brandcount_supplierlist
0,2,9,1
1,4,7,1
2,5,2,1
3,6,2,0
4,7,2,0


In [3]:
manufacturer_sql = """
SELECT id_manufacturer AS manufacturer_id
FROM manufacturer AS m;
"""
m_bf = helper.query_retrieval(manufacturer_sql, manufacturer_access)
b_df = pd.merge(m_bf, b_df, on = 'manufacturer_id', how='left')
b_df[:10]

,manufacturer_id,brandcount_watchlist,brandcount_supplierlist
0,353,NaN,NaN
1,458,NaN,NaN
2,402,NaN,NaN
3,369,NaN,NaN
4,311,1.0,0.0
5,476,NaN,NaN
6,1,NaN,NaN
7,3,NaN,NaN
8,121,NaN,NaN
9,967,NaN,NaN


#### brand# and received message#

In [4]:
message_sql = """
--The number of manufacturer received message coming from buyer
--The number of buyer who sent the message to manufacturer
WITH message AS (SELECT
                  c.id_conversation AS id_subject
                , c.subject 
                , c.fk_status AS fk_subject_status
                , cs.name AS subject_status
                , c.created_at AS subject_created_at
                , to_char(c.created_at::timestamp WITH TIME ZONE, 'iyyy"W"IW'::text) AS subject_created_year_week
                , to_char(c.created_at::timestamp WITH TIME ZONE, 'yyyy"M"mm'::text) AS subject_created_year_month
                , to_char(c.created_at::timestamp WITH TIME ZONE, 'yyyy'::text) AS subject_created_year
                , c.owner AS subject_owner
                , cp.id_conversation_participant AS id_subject_participant
                , cp.user_hash
                , cp.first_name 
                , cp.last_name
                , CASE WHEN fk_user_type = 1 THEN 'Buyer' WHEN fk_user_type = 2 THEN 'Manufacturer' ELSE 'CRM' END AS user_entity_type
                , cp.entity_id
                , cp.entity_name
                , cps.name AS participant_status
                , cp.is_admin AS is_subject_admin
                , cp.archived AS participant_is_archived
                , cp.is_online AS participant_is_online
                , cp.created_at AS participant_created_at
                , to_char(cp.created_at::timestamp WITH TIME ZONE, 'iyyy"W"IW'::text) AS participant_created_year_week
                , to_char(cp.created_at::timestamp WITH TIME ZONE, 'yyyy"M"mm'::text) AS participant_created_year_month
                , to_char(cp.created_at::timestamp WITH TIME ZONE, 'yyyy'::text) AS participant_created_year
                , m.id_conversation_participant_message AS id_message
                , m.message
                , mt.name AS message_type
                , ms.name AS message_status
                , m.link AS message_link
                , m.updated AS message_is_updated
                , m.created_at AS message_created_at
                , to_char(m.created_at::timestamp WITH TIME ZONE, 'iyyy"W"IW'::text) AS message_created_year_week
                , to_char(m.created_at::timestamp WITH TIME ZONE, 'yyyy"M"mm'::text) AS message_created_year_month
                , to_char(m.created_at::timestamp WITH TIME ZONE, 'yyyy'::text) AS message_created_year
                , m.sent_at AS message_sent_at
        FROM conversation AS c
        LEFT JOIN conversation_status AS cs 
                ON cs.id_conversation_status = c.fk_status
        LEFT JOIN conversation_participant AS cp 
               ON cp.fk_conversation = c.id_conversation
        LEFT JOIN conversation_participant_status AS cps 
               ON cps.id_conversation_participant_status = cp.fk_participant_status
        LEFT JOIN conversation_participant_message AS m 
               ON m.fk_conversation_participant = cp.id_conversation_participant
        LEFT JOIN conversation_participant_message_type AS mt 
               ON mt.id_conversation_participant_message_type = m.fk_conversation_participant_message_type
        LEFT JOIN conversation_participant_message_status AS ms
               ON ms.id_conversation_participant_message_status = m.fk_conversation_participant_message_status
        WHERE mt.name != 'Status'
        ORDER BY c.id_conversation, m.id_conversation_participant_message
)


SELECT  
        t.entity_uid,
        MIN(t.entity_id) AS manufacturer_id,
        COUNT(DISTINCT m3.user_entity_type || '-' || m3.entity_id) AS nr_buyers,
        COUNT(DISTINCT m3.id_message) AS nr_messages_received_from_buyers
FROM (
        SELECT  
                m.id_subject,
                m.entity_id,
                m.user_entity_type,
                m.user_entity_type || '-' || m.entity_id AS entity_uid,
                m.id_message,
                m.message
        FROM message AS m
) AS t
LEFT JOIN message AS m3 ON m3.id_subject = t.id_subject AND m3.user_entity_type = 'Buyer'
WHERE t.user_entity_type = 'Manufacturer'
GROUP BY t.entity_uid
ORDER BY 2;
"""
message_sql = helper.query_retrieval(message_sql, messaging_access)
message_sql[:10]

,entity_uid,manufacturer_id,nr_buyers,nr_messages_received_from_buyers
0,Manufacturer-4,4,1,64
1,Manufacturer-7,7,1,2
2,Manufacturer-8,8,0,0
3,Manufacturer-11,11,0,0
4,Manufacturer-12,12,3,4
5,Manufacturer-15,15,1,2
6,Manufacturer-18,18,0,0
7,Manufacturer-19,19,1,1
8,Manufacturer-23,23,0,0
9,Manufacturer-33,33,2,6


#### Calcuate manufacturer profile view and find distinct buyer number

In [5]:
pv_sql = """
-- manufacturer profile view number 
-- ? number of brand view manufacturer (need external join)
WITH pv_view AS (

SELECT 
        --DISTINCT ON (tp_ts.viewed_date, tp_ts.viewer_id_hashed, tp_ts.id_tracking_profile_tab),
        COUNT(tp.viewed_entity_id) OVER (PARTITION BY tp.viewed_entity_id) AS pv_count
        , tp.id_tracking_profile AS id_tracking_profile_tab
        , tp.viewer_hashed_id AS viewer_id_hashed
        , tp.viewed_entity_id AS viewed_id_entity
        , tp.fk_tracking_section
        , ts.name AS profile_tab
        , et.id_entity_type AS viewed_id_entity_type
        , et.name AS viewed_entity_type
        , tp.created_at AS viewed_datetime
        , tp.created_at::date AS viewed_date    
        , to_char(tp.created_at::timestamp with time zone, 'iyyy"W"IW'::text) AS viewed_year_week
        , to_char(tp.created_at::timestamp with time zone, 'yyyy"M"mm'::text) AS viewed_year_month
        , to_char(tp.created_at::timestamp with time zone, 'yyyy'::text) AS viewed_year
     
FROM public.tracking_profile AS tp
LEFT JOIN tracking_section AS ts ON tp.fk_tracking_section = ts.id_tracking_section     
LEFT JOIN entity_type AS et ON et.id_entity_type = ts.fk_entity_type
WHERE et.name = 'manufacturer'

)

SELECT pv.viewed_id_entity AS manufacturer_id,
       Min(pv_count) AS pv_count,
       pv.viewer_id_hashed AS uid
FROM pv_view AS pv
GROUP BY pv.viewed_id_entity,pv.viewer_id_hashed
ORDER BY pv.viewed_id_entity

"""
pv_sql = helper.query_retrieval(pv_sql, logging_access)
pv_sql[:5]

,manufacturer_id,pv_count,uid
0,2,87,9c73000b25f2c1e4ab2dad090db671e4
1,2,87,ef9e5846fc492f1afcf0c8e935375f67
2,2,87,b1936c5aded86a2113472b62650be9c7
3,2,87,87e912d5654a5627a0f729f949abcf26
4,2,87,d1ea6d09cc3a094c9e8d2814f8fbae81


#### pageview and distince buyer count

In [6]:
auth_sql = """
SELECT u.hashed_id AS uid,
       u.fk_entity::int AS buyer_entity
FROM public.users AS u
ORDER BY u.fk_entity;
"""
auth_sql = helper.query_retrieval(auth_sql, auth_access)
auth_sql[:5]

,uid,buyer_entity
0,5e227698d39203f3560aa4c27c2f8129,1.0
1,2179e2012ca43b09112edaa80f2cb832,2.0
2,546a5c5a0e05efb04c7fdcadaebed9c0,3.0
3,64372abfaf88c24db29fc3fd8e89a7a3,4.0
4,dc47fbc938fcbbc935ec7b22add13877,5.0


In [7]:
# left join the two table then count distinct company(or buyer) number
pv_Com_count = pd.merge(pv_sql, auth_sql, on='uid', how = 'left')
pv_Com_count[:10]

#Using groupby function to get the number of 'buyer_entity' 
pv_Com_count = pv_Com_count.groupby('manufacturer_id')['buyer_entity'].nunique()
#to_frame function can transfer pandas.core.series.Series to pandas.core.frame.DataFrame
pv_Com_count = pv_Com_count.to_frame().reset_index()


In [8]:
#Join all the SQL table together
total = pd.merge(pv_Com_count, message_sql, on = 'manufacturer_id', how='outer')
total = pd.merge(b_df, total, on = 'manufacturer_id', how='left')  
total = total.rename(index = str, columns = {"buyer_entity":"nr_viewer_buyer","nr_buyers":"nr_buyers_sent_message", "brandcount_watchlist":"nr_brand_watchlist","brandcount_supplierlist":"nr_brand_supplierlist" })

In [9]:
#drop useless field and get the final result of Y value

pv_sql_ = pv_sql.drop( ['uid'] , axis = 1).groupby('manufacturer_id')['pv_count'].min()
pv_sql_  = pv_sql_.to_frame().reset_index()
total = total.drop( ['entity_uid'] ,axis = 1)
total = pd.merge(total, pv_sql_, on = 'manufacturer_id', how='outer')

#set all Nan value to 0
total[total.isnull()] = 0

#remove the manufacturer_id
labels = total.drop(['manufacturer_id'], axis=1)
labels.shape

(1300, 6)

(1300, 1)

### export the data as label.csv

In [10]:
writer = pd.ExcelWriter('labels.xlsx')
labels.to_excel(writer,'label')
writer.save()